Build baseline model using Neural Network  

In [28]:
import pandas as pd

from google.colab import drive
drive.mount("/content/gdrive/")

data = pd.read_csv('/content/gdrive/My Drive/Projects/Car Insurance Pricing/training.csv')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [135]:
import numpy as np
import sklearn as sk

# divide to train and validation datasets, years 1-3 and validation on year 4
train_bronze = data[data["year"] < 4]
val_bronze = data[data["year"] < 4]


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

def data_prep_NN(data):

  """
  Prepare data to feed in Neural Network model"""
  
  # predictor columns to be included in model, factors onehotencoded, numericals scaled
  one_hot_X = ["pol_coverage", "pol_payd", "pol_usage", "drv_sex1", "vh_type"]
  scaler_X = ["pol_no_claims_discount","pol_duration", "drv_age1","vh_age","population"]


  # Onehotencoding
  oh_encoder = OneHotEncoder(handle_unknown='ignore').fit(data[one_hot_X])
  enc_silver = pd.DataFrame(oh_encoder.transform(data[one_hot_X]).toarray())
  enc_silver.columns = oh_encoder.get_feature_names(one_hot_X)

  # Scaling
  scaled_silver = MinMaxScaler().fit_transform(train_X[scaler_X])
  scaled_silver = pd.DataFrame(scaled_silver, columns=[scaler_X])
  scaled_silver.columns = scaler_X

  # Target column, "claim_amount"
  target_Y = data["claim_amount"]

  # pull data together
  data_gold = pd.concat([enc_silver,scaled_silver, target_Y], axis = 1).dropna()
  data_gold_Y = train_gold["claim_amount"]
  data_gold_X = train_gold.drop("claim_amount", axis= 1)
  
  return data_gold_X, data_gold_Y


train_X, train_Y = data_prep_NN(train_bronze)
val_X, val_Y = data_prep_NN(val_bronze)

In [136]:
# Create Neural network to predict regression

from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=112, activation='tanh', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=5000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)

base_NN_model = mlp.fit(train_X, train_Y)

In [137]:
# Evaluate RMSE 
from sklearn.metrics import mean_squared_error

pred_val = base_NN_model.predict(val_X)
rmse = np.sqrt(mean_squared_error(val_Y, pred_val))
rmse

576.1737012504997